In [1]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold

#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset1.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[1, 2,17,18,19])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)

In [2]:
ii = 1
xgb_model = XGBClassifier(random_state=0 ,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['feature_importance_vals']).to_string(index=False))
	ii += 1

Iteration 1
 feature_importance_vals
                0.060247
                0.314494
                0.696728
                0.245097
                0.930467
                0.067753
                0.112187
                0.299177
                0.079845
                0.031436
                0.167706
                0.049561
                0.051414
                0.211407
                0.289931
                0.045310
                0.519553
                0.446603
                0.362661
                0.451281
Iteration 2
 feature_importance_vals
                0.061886
                0.250457
                0.588861
                0.242293
                0.974767
                0.050528
                0.101153
                0.318589
                0.079587
                0.016162
                0.153618
                0.052210
                0.041747
                0.177695
                0.250337
                0.042545
                0.530057
 

In [3]:
ii = 1
dt_model=DecisionTreeClassifier(random_state=0,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
cv = StratifiedKFold(n_splits=10)

for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['feature_importance_vals']).to_string(index=False))
	ii += 1

Iteration 1
 feature_importance_vals
                0.000000
                0.072221
                0.121322
                0.052641
                0.154146
                0.003213
                0.010782
                0.011184
                0.000000
                0.042943
                0.017180
                0.000000
                0.007709
                0.017835
                0.020165
                0.000000
                0.097255
                0.024817
                0.000000
                0.004034
Iteration 2
 feature_importance_vals
                0.006204
                0.056407
                0.110251
                0.051491
                0.163057
                0.000000
                0.006857
                0.008565
                0.003343
                0.043933
                0.013228
                0.002129
                0.006022
                0.000000
                0.017009
                0.015382
                0.100860
 